In [49]:
import pandas as pd
import os
from datetime import datetime, timedelta
import talib
import pandas as pd
import numpy as np
import cufflinks as cf
from plotly.offline import init_notebook_mode, iplot
import plotly.express as px
import plotly.graph_objs as go
import warnings
import enum
warnings.filterwarnings('ignore')
init_notebook_mode(connected=True) 
import sqlite3

In [50]:
tradingList = {
    "HINDPETRO": {"price": 262.25, "leverage": 10},
    "HINDZINC": {"price": 317.10, "leverage": 10},
    "SBIN": {"price": 364.65, "leverage": 9},
    "ICICIBANK": {"price": 594.75, "leverage": 9},
    "WIPRO": {"price": 507.60, "leverage": 9},
    "ITC": {"price": 203.25, "leverage": 9},
    "BHARTIARTL": {"price": 561.75, "leverage": 9},
    "IOC": {"price": 102.80, "leverage": 9},
    "ONGC": {"price": 115.10, "leverage": 9},
    "DABUR": {"price": 529.20, "leverage": 9},
    "GAIL": {"price": 161.05, "leverage": 9},
    "TATACONSUM": {"price": 640.05, "leverage": 9},
    "AMBUJACEM": {"price": 309.05, "leverage": 9},
    "DLF": {"price": 265.30, "leverage": 8},
    "TATAPOWER": {"price": 109.35, "leverage": 8},
    "TATAMOTORS": {"price": 315.55, "leverage": 7}
}

tickerList = list(tradingList.keys())
db_hammer = "E:\\Workspace\\Python\\PythonPOC\\notebook\\data\\db\\hammer\\statistics.db"
data_directory = "E:\\Workspace\\Python\\PythonPOC\\notebook\\data\\test_data\\hammer\\"


In [51]:
class Trend(enum.Enum):
    strong_uptrend = 1
    uptrend = 2
    strong_downtrend = 3
    downtrend = 4
    uptrend_reversal = 5
    downtrend_reversal = 6
    strong_uptrend_reversal = 7
    strong_downtrend_reversal = 8
    sidetrend = 9

In [52]:
class Candle(enum.Enum):
    red = 1
    green = 2
    doji = 3

In [53]:
class CandleHelper:

    def __init__(self, count=4):
        self.count = count
        self.index = 0

    def is_doji_candle(self, candle):
        return candle == Candle.doji

    def is_green_candle(self, candle):
        return candle == Candle.green

    def is_red_candle(self, candle):
        return candle == Candle.red

    def get_last_candle(self, data):
        if data['close'].values > data['open'].values:
            return Candle.green
        if data['close'].values < data['open'].values:
            return Candle.red
        return Candle.doji

    def get_candle_details(self, data):
        candle_data = data.tail(4)
        current = self.get_last_candle(candle_data.tail(1))
        previous = self.get_last_candle(candle_data.tail(2).head(1))
        last_two = self.get_last_candle(candle_data.tail(3).head(1))
        last_three = self.get_last_candle(candle_data.head(1))
        return current, previous, last_two, last_three

    def trend_strength(self, data, trend):
        candle_data = data.tail(4)
        current = candle_data.tail(1)
        previous = candle_data.tail(2).head(1)
        last_two = candle_data.tail(3).head(1)
        last_three = candle_data.head(1)
        reversal_value = 0
        if trend == Trend.strong_uptrend:
            trend_value = abs(round((last_three['close'].values - current['open'].values)[0], 2))
        elif trend == Trend.uptrend:
            trend_value = abs(round((last_two['close'].values - current['open'].values)[0], 2))
        elif trend == Trend.strong_downtrend:
            trend_value = abs(round((last_three['close'].values - current['open'].values)[0], 2))
        elif trend == Trend.downtrend:
            trend_value = abs(round((last_two['close'].values - current['open'].values)[0], 2))
        elif trend == Trend.uptrend_reversal:
            trend_value = abs(round((last_two['close'].values - previous['open'].values)[0], 2))
            reversal_value = abs(round((current['close'].values - previous['open'].values)[0], 2))
        elif trend == Trend.downtrend_reversal:
            trend_value = abs(round((last_two['close'].values - previous['open'].values)[0], 2))
            reversal_value = abs(round((current['close'].values - previous['open'].values)[0], 2))
        elif trend == Trend.strong_uptrend_reversal:
            trend_value = abs(round((last_three['close'].values - previous['open'].values)[0], 2))
            reversal_value = abs(round((current['close'].values - previous['open'].values)[0], 2))
        elif trend == Trend.strong_downtrend_reversal:
            trend_value = abs(round((last_three['close'].values - previous['open'].values)[0], 2))
            reversal_value = abs(round((current['close'].values - previous['open'].values)[0], 2))
        else:
            trend_value = 0
            reversal_value = 0
        return trend_value, reversal_value

    def get_trends(self, data):
        current, previous, last_two, last_three = self.get_candle_details(data)
        trend = Trend.sidetrend
        if (self.is_green_candle(current) or self.is_doji_candle(current)) and (
                self.is_green_candle(previous) or self.is_doji_candle(previous)):
            if self.is_green_candle(last_two) or self.is_doji_candle(last_two):
                if self.is_green_candle(last_three) or self.is_doji_candle(last_three):
                    trend = Trend.strong_uptrend
                else:
                    trend = Trend.uptrend
        elif (self.is_red_candle(current) or self.is_doji_candle(current)) and (
                self.is_red_candle(previous) or self.is_doji_candle(previous)):
            if self.is_red_candle(last_two) or self.is_doji_candle(last_two):
                if self.is_red_candle(last_three) or self.is_doji_candle(last_three):
                    trend = Trend.strong_downtrend
                else:
                    trend = Trend.downtrend
        elif self.is_red_candle(current) and (self.is_green_candle(previous) or self.is_doji_candle(previous)):
            if self.is_green_candle(last_two) or self.is_doji_candle(last_two):
                if self.is_green_candle(last_three) or self.is_doji_candle(last_three):
                    trend = Trend.strong_uptrend_reversal
                else:
                    trend = Trend.uptrend_reversal
        elif self.is_green_candle(current) and (self.is_red_candle(previous) or self.is_doji_candle(previous)):
            if self.is_red_candle(last_two) or self.is_doji_candle(last_two):
                if self.is_red_candle(last_three) or self.is_doji_candle(last_three):
                    trend = Trend.strong_downtrend_reversal
                else:
                    trend = Trend.downtrend_reversal
        else:
            trend = Trend.sidetrend
        trend_value, reversal_value = self.trend_strength(data, trend)
        return trend, trend_value, reversal_value

    def get_next_candle(self,data):
        candle_data = data.tail(1)
        self.index +=1
        return self.get_last_candle(candle_data)

In [88]:
def candle_graph(data,ticker):
    Candle = go.Candlestick(x=data.index,
                       open=data.open,
                       high=data.high,
                       low=data.low,
                       close=data.close
                       )  
    
                   
    iplot([Candle])

In [121]:
def graph(data,ticker):    
    pattern_data = pd.DataFrame(columns=['x','Marker','Symbol','Color'])   
    start = pd.DataFrame(columns=['x','Marker','Symbol','Color'])
    end = pd.DataFrame(columns=['x','Marker','Symbol','Color'])
    helper = CandleHelper()    
    next_long_pattern = False
    trade_open = False
    current_price = 0
    for index, row in data.iterrows():
        start_data = {} 
        end_data = {} 
        if trade_open:
            candle_data = data.loc[index:index]
            closing_price = candle_data['close'].values[0] 
            spread = round(abs(candle_data['open'].values[0]  - closing_price),2)
            pattern = helper.get_next_candle(candle_data)
            if (pattern == Candle.red and spread >= 0.15 ) or closing_price <= current_price - 1:
                end_data["x"] = index
                end_data["Marker"] = row["close"]
                end_data["Symbol"] ="star-triangle-down"
                end_data["Color"] ="red" 
                end = end.append(end_data, ignore_index=True) 
                trade_open = False
        if next_long_pattern:
            if helper.index > 3:
                helper.index = 0
                next_long_pattern = False            
            else:
                pattern = helper.get_next_candle(data.loc[index:index])
                if pattern == Candle.green:
                    helper.index = 0
                    next_long_pattern = False
                    start_data["x"] = index
                    start_data["Marker"] = row["close"]
                    start_data["Symbol"] ="star-triangle-up"
                    start_data["Color"] ="green"
                    start = start.append(start_data, ignore_index=True) 
                    current_price = row["close"]
                    trade_open = True
        else:
            ticker_data = {}        
            val = row["Hammer"]
            if val == 100:                             
                ticker_data["x"] = index
                ticker_data["Marker"] = row["close"]
                ticker_data["Symbol"] ="circle"
                ticker_data["Color"] ="gold"
                pattern_data = pattern_data.append(ticker_data, ignore_index=True) 

                current_data = data.loc[index-3:index]
                trend, trend_value, reversal_value = helper.get_trends(current_data)
                if trend == Trend.downtrend_reversal or trend == Trend.strong_downtrend_reversal:
                    start_data["x"] = index
                    start_data["Marker"] = row["close"]
                    start_data["Symbol"] ="star-triangle-up"
                    start_data["Color"] ="green"
                    start = start.append(start_data, ignore_index=True) 
                    current_price = row["close"]
                    trade_open = True
                else:
                    helper.index = 0
                    next_long_pattern = True
            
            
    CandleStick = go.Candlestick(x=data.index,
                       open=data.open,
                       high=data.high,
                       low=data.low,
                       close=data.close
                       )  
    Pattern = go.Scatter(x=pattern_data.x,
                       y=pattern_data.Marker,
                       mode='markers',
                       name ='markers',
                       marker=go.Marker(size=20,
                                        symbol=pattern_data["Symbol"],
                                        color=pattern_data["Color"]))
    
    StartData = go.Scatter(x=start.x,
               y=start.Marker,
               mode='markers',
               name ='markers',
               marker=go.Marker(size=20,
                                symbol=start["Symbol"],
                                color=start["Color"]))
    
    
    EndData = go.Scatter(x=end.x,
                   y=end.Marker,
                   mode='markers',
                   name ='markers',
                   marker=go.Marker(size=20,
                                    symbol=end["Symbol"],
                                    color=end["Color"]))
                         
    iplot([CandleStick, Pattern,StartData,EndData])

In [106]:
class StatisticsManager:

    def __init__(self,name):    
        self.name =name  
        self.db = sqlite3.connect(self.name)
        

    def connectDB(self):
        self.db = sqlite3.connect(self.name)
        
    def get_trade_all(self):
        self.connectDB()
        query = '''SELECT * FROM Statistics'''        
        data = pd.read_sql(query, con=self.db)
        self.close()
        return data
    
    def get_trade_day(self, day):
        self.connectDB()
        query = '''SELECT close_time,current,close,number_of_shares,symbol,type,sum(case when  status = 'loss' then 1 else 0 end) as loss,
                       sum(case when status = 'profit' then 1 else 0 end) as profit,
                       sum(case when status = 'open' then 1 else 0 end) as open,
                       round(sum(case when  status = 'loss' then 1.0 else 0.0 end)/count(*),2) as loss_ratio,
                       round(sum(case when status = 'profit' then 1.0 else 0.0 end)/count(*),2)  as profit_ratio,
                       round(sum(close*number_of_shares -current*number_of_shares ),2)  as returns
                    FROM Statistics GROUP by symbol,close_time having close_time = '''
        query += "'" + day + "'"
        query += ''' ORDER by returns'''
        data = pd.read_sql(query, con=self.db)
        self.close()
        return data

    def get_trade_loss(self):
        self.connectDB()
        query = '''SELECT close_time, sum(case when  status = 'loss' then 1 else 0 end) as loss,
                       sum(case when status = 'profit' then 1 else 0 end) as profit,
                       sum(case when status = 'open' then 1 else 0 end) as open,
                       round(sum(case when  status = 'loss' then 1.0 else 0.0 end)/count(*),2) as loss_ratio,
                       round(sum(case when status = 'profit' then 1.0 else 0.0 end)/count(*),2)  as profit_ratio,
                       round(sum(close*number_of_shares -current*number_of_shares ),2)  as returns
                    FROM Statistics GROUP by close_time having returns < 0  ORDER by returns'''        
        data = pd.read_sql(query, con=self.db)
        self.close()
        return data
    

    def get_trade_close_time(self):
        self.connectDB()
        query = '''SELECT close_time, sum(case when  status = 'loss' then 1 else 0 end) as loss,
                       sum(case when status = 'profit' then 1 else 0 end) as profit,
                       sum(case when status = 'open' then 1 else 0 end) as open,
                       round(sum(case when  status = 'loss' then 1.0 else 0.0 end)/count(*),2) as loss_ratio,
                       round(sum(case when status = 'profit' then 1.0 else 0.0 end)/count(*),2)  as profit_ratio,
                       round(sum(close*number_of_shares -current*number_of_shares ),2)  as returns
                    FROM Statistics GROUP by close_time ORDER by returns'''        
        data = pd.read_sql(query, con=self.db)
        self.close()
        return data
    
    def get_trade_algorithm(self):
        self.connectDB()
        query = '''SELECT algorithm,sum(case when  status = 'loss' then 1 else 0 end) as loss,
                       sum(case when status = 'profit' then 1 else 0 end) as profit,
                       sum(case when status = 'open' then 1 else 0 end) as open,
                       round(sum(case when  status = 'loss' then 1.0 else 0.0 end)/count(*),2) as loss_ratio,
                       round(sum(case when status = 'profit' then 1.0 else 0.0 end)/count(*),2)  as profit_ratio,
                       round(sum(close*number_of_shares -current*number_of_shares ),2)  as returns
                    FROM Statistics GROUP by algorithm ORDER by profit_ratio desc'''        
        data = pd.read_sql(query, con=self.db)
        self.close()
        return data
    
    def get_trade_symbol(self):
        self.connectDB()
        query = '''SELECT symbol,type,sum(case when  status = 'loss' then 1 else 0 end) as loss,
                       sum(case when status = 'profit' then 1 else 0 end) as profit,
                       sum(case when status = 'open' then 1 else 0 end) as open,
                       round(sum(case when  status = 'loss' then 1.0 else 0.0 end)/count(*),2) as loss_ratio,
                       round(sum(case when status = 'profit' then 1.0 else 0.0 end)/count(*),2)  as profit_ratio,
                       round(sum(close*number_of_shares -current*number_of_shares ),2)  as returns
                    FROM Statistics GROUP by symbol,type ORDER by profit_ratio desc'''        
        data = pd.read_sql(query, con=self.db)
        self.close()
        return data


    def close(self):
        self.db.close()

    def deleteDB(self,db):
        if os.path.exists(db):
            os.remove(db)
        else:
            print("The file does not exist")

In [107]:
def get_all_loss():
    manager = StatisticsManager(db_hammer)
    loss_data = manager.get_trade_loss()
    trade_data = pd.DataFrame(columns = ['close_time', 'loss', 'profit','open','loss_ratio','profit_ratio','returns'])
    date_list = loss_data["close_time"].unique()
    for day in date_list:
        row_data = manager.get_trade_day(day)
        trade_data = trade_data.append(row_data, ignore_index=True)
    return date_list,trade_data
    

In [108]:
date_list,loss_data = get_all_loss()

In [109]:
loss_data

,close_time,loss,profit,open,loss_ratio,profit_ratio,returns,current,close,number_of_shares,symbol,type
0,2021-05-09-11:44:13,1,0,0,1.0,0.0,-1896.00,269.10,257.25,160.0,DLF,long
1,2021-05-09-11:44:13,1,0,0,1.0,0.0,-1128.00,446.75,434.75,94.0,WIPRO,long
2,2021-05-09-11:44:13,1,0,0,1.0,0.0,-1002.15,337.90,330.25,131.0,SBIN,long
3,2021-05-09-11:44:13,1,0,0,1.0,0.0,-811.20,101.00,99.05,416.0,ONGC,long
4,2021-05-09-11:44:13,1,0,0,1.0,0.0,-793.10,307.50,302.35,154.0,AMBUJACEM,long
...,...,...,...,...,...,...,...,...,...,...,...,...
263,2021-05-22-11:44:13,0,1,0,0.0,1.0,252.45,136.40,137.25,297.0,GAIL,long
264,2021-05-22-11:44:13,0,1,0,0.0,1.0,345.15,288.00,290.95,117.0,TATAMOTORS,long
265,2021-05-22-11:44:13,0,1,0,0.0,1.0,352.00,244.75,246.95,160.0,DLF,long
266,2021-05-22-11:44:13,0,1,0,0.0,1.0,508.50,541.80,547.45,90.0,DABUR,long


In [110]:
loss_data.loc[263:265]

,close_time,loss,profit,open,loss_ratio,profit_ratio,returns,current,close,number_of_shares,symbol,type
263,2021-05-22-11:44:13,0,1,0,0.0,1.0,252.45,136.40,137.25,297.0,GAIL,long
264,2021-05-22-11:44:13,0,1,0,0.0,1.0,345.15,288.00,290.95,117.0,TATAMOTORS,long
265,2021-05-22-11:44:13,0,1,0,0.0,1.0,352.00,244.75,246.95,160.0,DLF,long


In [111]:
filename = data_directory+"DLF.csv"
data = pd.read_csv(filename)

In [112]:
data.columns

Index(['Unnamed: 0', 'open', 'high', 'low', 'close', 'volume', 'traded',
       'Hammer', 'ticker', 'date'],
      dtype='object')

In [113]:
data = data.drop('Unnamed: 0',axis=1)

In [114]:
hammer = data[data['Hammer']!=0].dropna()

In [115]:
loss_data.loc[263:263]

,close_time,loss,profit,open,loss_ratio,profit_ratio,returns,current,close,number_of_shares,symbol,type
263,2021-05-22-11:44:13,0,1,0,0.0,1.0,252.45,136.4,137.25,297.0,GAIL,long


In [116]:
loss_data.loc[263:265]
data_list =[]
for index, row in hammer.iterrows():
    data_list.append(data.loc[index-4:index+14])
data_list       

[      open    high     low   close   volume  traded  Hammer ticker  \
 80  292.30  292.45  292.00  292.30  33517.0   False     0.0    DLF   
 81  292.20  292.90  292.10  292.90  18731.0   False     0.0    DLF   
 82  292.75  293.20  292.75  293.00  12962.0   False     0.0    DLF   
 83  293.10  293.40  292.95  292.95  11818.0   False     0.0    DLF   
 84  292.95  293.00  292.75  293.00  10235.0    True   100.0    DLF   
 85  292.90  293.00  292.25  292.35  11688.0   False     0.0    DLF   
 86  292.35  292.55  292.10  292.20  12183.0   False     0.0    DLF   
 87  292.20  292.30  291.70  292.15  50637.0   False     0.0    DLF   
 88  292.30  292.55  292.05  292.35  10644.0   False     0.0    DLF   
 89  292.45  292.70  292.35  292.65   6395.0   False     0.0    DLF   
 90  292.65  293.25  292.60  293.05  19581.0   False     0.0    DLF   
 91  293.05  293.60  293.00  293.30  28599.0   False     0.0    DLF   
 92  293.30  293.45  293.05  293.25   4655.0   False     0.0    DLF   
 93  2

In [127]:
graph(data_list[13],"DLF")


In [97]:
candle_graph(data_list[0],"DLF")